In [3]:
import pandas as pd



In [5]:
# Load raw data
df = pd.read_csv("/Users/mm/vantage/raw_stock_data.csv", index_col="date")
df = df.astype(float)  # Ensure numeric columns
print(df.head())

              open      high       low   close      volume
date                                                      
2025-02-19  244.66  246.0100  243.1604  244.87  32204215.0
2025-02-18  244.15  245.1800  241.8400  244.47  48822491.0
2025-02-14  241.25  245.5500  240.9900  244.60  40896227.0
2025-02-13  236.91  242.3399  235.5700  241.53  53614054.0
2025-02-12  231.20  236.9600  230.6800  236.87  45243292.0


In [7]:
# Transform
try:
    df["volatility"] = (df["high"] - df["low"]) / df["close"]
    # Handle missing/null values
    df.dropna(inplace=True)
    print("Volatility calculated!")
except Exception as e:
    print(f"Transformation error: {e}")
    raise

Volatility calculated!


In [11]:
# Clean up
df = df[["open", "close", "volatility"]]  # Keep relevant columns
df.index = pd.to_datetime(df.index)  # Ensure date format
df.to_csv("/Users/mm/vantage/transformed_stock_data.csv")
print("Data transformed and saved!")

Data transformed and saved!


In [13]:
df_transformed = pd.read_csv("/Users/mm/vantage/transformed_stock_data.csv", index_col="date")
df_transformed = df.astype(float)  # Ensure numeric columns
print(df_transformed.head())

              open   close  volatility
date                                  
2025-02-19  244.66  244.87    0.011637
2025-02-18  244.15  244.47    0.013662
2025-02-14  241.25  244.60    0.018643
2025-02-13  236.91  241.53    0.028029
2025-02-12  231.20  236.87    0.026512


In [15]:
import sqlite3

# Connect to SQLite
conn = sqlite3.connect("trading_data.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS stock_data (
        date TEXT PRIMARY KEY,
        open REAL,
        close REAL,
        volatility REAL
    )
""")

In [17]:
# Load data into SQLite
df.to_sql("stock_data", conn, if_exists="replace", index=True)
conn.commit()
print("Data loaded into SQLite!")

Data loaded into SQLite!


In [19]:
# Verify
result = pd.read_sql("SELECT * FROM stock_data LIMIT 5", conn)
print(result)
conn.close()

                  date    open   close  volatility
0  2025-02-19 00:00:00  244.66  244.87    0.011637
1  2025-02-18 00:00:00  244.15  244.47    0.013662
2  2025-02-14 00:00:00  241.25  244.60    0.018643
3  2025-02-13 00:00:00  236.91  241.53    0.028029
4  2025-02-12 00:00:00  231.20  236.87    0.026512
